In [1]:
import pandas as pd
import numpy as np
import sys
import os
from db import connection_db, close_connection

# Code to acccess data in server
conn = connection_db('35.187.71.239', 'pri', 'rampup2019', 'kremlin_koder')
# Retrieving all scraped data from the LinkedIn profiles and putting them in a pandas dataframe
'''  The url and the name were scraped for testing and verification purposes.
 They will not be obtained in the final product due to privacy policy (GDPR).
'''
#query = """
#SELECT e.* FROM pri.employee as e JOIN pri.google_urls g ON e.url = g.url WHERE g.query = "{}";
#""".format(search)

query = """
SELECT e.* FROM pri.employee as e;
"""
employee = pd.read_sql(query, conn)

# Sets MySQL_ID as the index for the employees table
employee.set_index('MySQL_ID', inplace = True)
employee['employee_ID'] = employee.index

# Retrieving information from each job position of each employee and putting them in a pandas dataframe
# Only previous job experiences are considered. Experiences that have End Date = "Present" are discarded.
#query = """
#SELECT exp.* FROM pri.experience as exp JOIN pri.employee emp ON exp.employee_ID = emp.MySQL_ID JOIN pri.google_urls g ON emp.MySQL_ID = g.MySQL_ID WHERE exp.Start IS NOT NULL AND exp.End IS NOT NULL AND exp.End <> "Present" AND g.query = "{}";
#""".format(search)

query = """
SELECT exp.* FROM pri.experience as exp JOIN pri.employee emp ON exp.employee_ID = emp.MySQL_ID WHERE exp.Start IS NOT NULL AND exp.End IS NOT NULL AND exp.End <> "Present";
"""


experience = pd.read_sql(query, conn)


# Retrieving information from each education degree of each employee and putting them in a pandas dataframe
#query = """
#SELECT edu.* FROM pri.education as edu JOIN pri.employee emp ON edu.employee_ID = emp.MySQL_ID JOIN pri.google_urls g ON emp.MySQL_ID = g.MySQL_ID WHERE edu.Start IS NOT NULL AND edu.End IS NOT NULL AND g.query = "{}";
#""".format(search)



query = """
SELECT edu.* FROM pri.education as edu JOIN pri.employee emp ON edu.employee_ID = emp.MySQL_ID WHERE edu.Start IS NOT NULL AND edu.End IS NOT NULL;
"""


education = pd.read_sql(query, conn)

# Closing connection to the database
close_connection(conn)

#remove present experience
experience = experience[experience.End!= "Present"]
#parsing start and end dates into datetime object
experience["Start"]= pd.to_datetime(experience['Start'])
experience['End']=pd.to_datetime(experience['End'])
result = pd.DataFrame()
# compute time spent in a role
experience['inRole_time'] = (experience['End'] - experience['Start']).map(lambda x: round(x.days/30))
# counts the number of experiences and add tham to the parent table EMP
# num_exp = experience['num_exp'] = experience.groupby(['employee_ID']).size()
employee['num_exp'] = experience.groupby(['employee_ID']).size()
# calculates the total time of experiences and add tham to the parent table EMP
employee['sum_exp'] = experience.groupby(['employee_ID']).sum()['inRole_time']
# calculates the averge time of experiences and add tham to the parent table EMP
employee['avg_exp'] = employee['sum_exp']/employee['num_exp'] 
# calculates time spent in one job role   - 2nd degree variable 
exp_in_comp = experience.groupby(['employee_ID','company']).sum()['inRole_time']
#calculate avg_time in company ( discussion - is average time the best representation of the person)
# employee['avg_timeInCompany']=exp_in_comp.groupby('employee_ID').mean()
# checks if the role was a promotion?
experience['isPromotion'] = experience.company == experience.company.shift(-1)
experience.isPromotion.replace({True:1,False:0}, inplace=True)
employee['Promotions'] = experience.groupby('employee_ID').sum()['isPromotion']
#calculate age
employee['age'] = 22+(employee['sum_exp']/12)

experience.drop(['MySQL_Date','description','position','location','Start','End'],axis=1,inplace=True)



education.End.replace({"Present": pd.to_datetime('today')}, inplace=True)


# Parses Start and End dates format to datetime for calculations
education["Start"]= pd.to_datetime(education['Start'])
education['End']=pd.to_datetime(education['End'])


# Calculates time spent in each degree (in months)
education['time_education'] = (education['End'] - education['Start']).map(lambda x: round(x.days/30))


# Calculates the total time of education and adds it to the employees df
employee['time_education'] = education.groupby(['employee_ID']).sum()['time_education']


# Counts the number of degrees and adds it to the employees df
employee['count_education'] = education.groupby(['employee_ID']).size()


# Replaces NaN values to mean in case no education information is provided in the scraped profile
employee.fillna(employee.mean(), inplace=True)

#merge experience and employee table to crete a table with one experience as one record
result = pd.merge(experience, employee, left_on='employee_ID', right_index=True,how='left', sort=False);
employee.reset_index(inplace = True)

# Removing unnecessary fields for fitting script
employee.drop(['MySQL_ID', 'MySQL_Date', 'url', 'name', 'currentposition'], axis=1, inplace = True)


result = pd.merge(experience, employee, left_on='employee_ID', right_on='employee_ID',how='left', sort=True);

#drop unnexessary columns
result.drop(['premium','connections','google_urls_id'],axis=1,inplace=True)

# adding inrole_time for consequtive time in one comapny
arr =  result[['employee_ID','inRole_time','isPromotion']].to_numpy()
arr = arr.transpose()
for i in range(arr.shape[1]):
#     print(arr[0][i])
    if arr[2][i] == 1 and arr[0][i] == arr[0][i+1]:
        arr[1][i] = arr[1][i] + arr[1][i+1]

result['inRole_time'] = arr[1]

#dropping role before promotion - post processiong duplicate 
rows  = result.loc[result['isPromotion']==1].index+1
result.drop(rows,inplace=True,errors='ignore')

#Calculate features for every experience in time
np.set_printoptions(threshold=sys.maxsize)
tocompute =  result[['employee_ID','num_exp','sum_exp','inRole_time','isPromotion','avg_exp','Promotions','age']].to_numpy()
tocompute = tocompute.transpose()
tocompute.astype(int,copy=False)

#Calculate employee data for particular job role
for i in range(tocompute.shape[1]-1):
     if tocompute[0][i] == tocompute[0][i+1]:
            if tocompute[4][i] == 1:
                tocompute[1][i] = tocompute[1][i] - 1

            tocompute[1][i+1] = tocompute[1][i] - 1
            tocompute[2][i+1] = tocompute[2][i] - tocompute[3][i]
            tocompute[6][i+1] = tocompute[6][i] - tocompute[4][i]
            tocompute[7][i+1] = tocompute[7][i] - (tocompute[3][i]/12)
result['num_exp'] = tocompute[1]
result['sum_exp'] = tocompute[2]
result['avg_exp'] =  tocompute[5]
result['Promotions'] = tocompute[6]
result['age'] = tocompute[7]


#calculate new avg experience
result['avg_exp'] = result['sum_exp']/result['num_exp'] 

#removing infinity,nan  and unnecessary fields
result.replace([np.inf, -np.inf], np.nan,inplace=True)
result.dropna(inplace=True)
result.drop(['company','employee_ID','query_id'],axis=1,inplace=True)
result.head()




# result['category']

# result.head(1).to_csv(r'./clean_data/employee1.csv', index = False)

,inRole_time,isPromotion,num_exp,sum_exp,avg_exp,Promotions,age,time_education,count_education
0,24,1,4.0,44.0,11.000000,1.0,25.666667,110.0,3.0
2,13,0,3.0,20.0,6.666667,0.0,23.666667,110.0,3.0
3,4,0,2.0,7.0,3.500000,0.0,22.583333,110.0,3.0
4,3,0,1.0,3.0,3.000000,0.0,22.250000,110.0,3.0
5,79,1,11.0,352.0,32.000000,4.0,51.333333,170.0,6.0


In [3]:
result = result[(result['inRole_time'] > 0)]
result = result[(result['inRole_time'] <= 60)]

(14696, 9)

In [6]:
result['category'] = np.ceil(result['inRole_time']/12)
result['category'].replace([4,5],3,inplace=True)
result.drop('inRole_time',axis=1,inplace=True)

,isPromotion,num_exp,sum_exp,avg_exp,Promotions,age,time_education,count_education,category
0,1,4.0,44.0,11.000000,1.0,25.666667,110.00000,3.0000,2.0
2,0,3.0,20.0,6.666667,0.0,23.666667,110.00000,3.0000,2.0
3,0,2.0,7.0,3.500000,0.0,22.583333,110.00000,3.0000,1.0
4,0,1.0,3.0,3.000000,0.0,22.250000,110.00000,3.0000,1.0
9,0,8.0,210.0,26.250000,2.0,39.500000,170.00000,6.0000,1.0
10,0,7.0,200.0,28.571429,2.0,38.666667,170.00000,6.0000,2.0
15,0,2.0,34.0,17.000000,0.0,24.833333,170.00000,6.0000,1.0
16,0,1.0,24.0,24.000000,0.0,24.000000,170.00000,6.0000,2.0
17,0,1.0,2.0,2.000000,0.0,22.166667,96.96682,3.0669,1.0
18,0,1.0,27.0,27.000000,0.0,24.250000,122.00000,3.0000,3.0


In [7]:
result.to_csv(r'./clean_data/employee_categorised.csv', index = False)